### init

In [2]:
import os 
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm

sectors = ['agriculture', 'apparel', 'chemicals', 'machinery', 'materials', 
           'metals', 'miscellaneous', 'transport', 'minerals']

new_sectors = ['Agriculture', 'Apparel', 'Chemicals', 'Machinery', 'Materials', 
           'Metals', 'Miscellaneous', 'Transport', 'Minerals']
rename_columns = {s: ns for s, ns in zip(sectors, new_sectors)}
sectors = new_sectors

region2agg_dict = {
    'east africa': 'Africa', 
    'central africa': 'Africa',
    'north africa': 'Africa',
    'southern africa': 'Africa',
    'west africa': 'Africa',
    'caribbean': 'America',
    'central america': 'America',
    'south america': 'America',
    'north america': 'America',
    'central asia': 'Asia',
    'east asia': 'Asia',
    'south asia': 'Asia',
    'southeast asia': 'Asia',
    'west asia': 'Asia',
    'eastern europe': 'Europe', 
    'nordic': 'Europe',
    'southern europe': 'Europe',
    'western europe': 'Europe',
    'australia and new zealand': 'Oceania',
    'melanesia': 'Oceania',
    'micronesia': 'Oceania',
    'polynesia': 'Oceania',   
}


project_dir = Path('C:\\shares_TARIFF_project\\comtrade_new\\DID_cancel2018')


volume_df = pd.read_csv(project_dir / '2015_2021.csv', header=0)
volume_df = volume_df.rename(columns=rename_columns)
volume_df.head()

,source,year,month,Agriculture,Apparel,Chemicals,Machinery,Materials,Metals,Minerals,Miscellaneous,Transport
0,Afghanistan,2015,1,807693.0,508764.0,NaN,45445.0,NaN,4956.0,NaN,269303.0,20858.0
1,Afghanistan,2015,2,564296.0,170889.0,NaN,47867.0,NaN,4838991.0,NaN,403538.0,NaN
2,Afghanistan,2015,3,153039.0,731135.0,NaN,5582.0,NaN,146989.0,NaN,279085.0,NaN
3,Afghanistan,2015,4,384263.0,281741.0,NaN,26328.0,2856.0,474628.0,NaN,1070675.0,4451.0
4,Afghanistan,2015,5,57945.0,240599.0,NaN,10655.0,24119.0,1664127.0,NaN,676760.0,3607.0


In [3]:
region_df = pd.read_csv(project_dir / 'countries2regions_new.csv', header=0)
country2region_dict = {c:r for c, r in zip(region_df['source_country'], region_df['source_region'])}

region_df['agg'] = region_df['source_region'].map(region2agg_dict)

region2countries_dict = {}
for region, group_df in region_df.groupby(by=['agg']):
    region2countries_dict[region] = group_df['source_country'].tolist()

# 区分 China和 Rest of Asia(ROA)
# china_list = ['Mainland China', 'Hong Kong', 'Taiwan']
china_list = ['China','China Macao SAR']
region2countries_dict['China'] = china_list
roa_list = []
for c in region2countries_dict.get('Asia'):
    if c not in china_list:
        roa_list.append(c)
region2countries_dict['ROA'] = roa_list
del region2countries_dict['Asia']


C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_1448\3596448589.py:7: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for region, group_df in region_df.groupby(by=['agg']):


### get panel data - trade war

In [5]:
country2region_dict = {}
for rg, ct_list in region2countries_dict.items():
    for ct in ct_list:
        country2region_dict[ct] = rg

# for sector in sectors:
#     volume_df[sector] = np.log(volume_df[sector] + 1)

volume_df['region'] = volume_df['source'].map(country2region_dict)
volume_sum_df = volume_df.groupby(by=['region', 'year', 'month']).sum()

volume_sum_df['region'], volume_sum_df['year'], volume_sum_df['month'] = zip(*volume_sum_df.index)
volume_sum_df.reset_index(drop=True, inplace=True)
#print(volume_sum_df.head())
#volume_sum_df.to_csv(r'C:\\Users\\Ki Siyuan\\Desktop\\vol_sum.csv')
# volume_mean_df = volume_sum_df.groupby(by=['year', 'month']).mean()
# volume_mean_df['year'], volume_mean_df['month'] = zip(*volume_mean_df.index)
# volume_mean_df.reset_index(drop=True, inplace=True)


# volume => percentage 转化为占比
years = volume_sum_df['year'].unique()
months = volume_sum_df['month'].unique()
for y in years:
    for m in months:
        query_df = volume_sum_df.query('year == @y and month == @m')[sectors]
        _sum = query_df.sum()
        secter2sum_dict = {s:v for s, v in zip(_sum.index, _sum.tolist())}
        for idx in query_df.index:
            for s, v in secter2sum_dict.items():
                _v = volume_sum_df.loc[idx, s]
                p = _v / v * 100
                volume_sum_df.loc[idx, s] = p
print(volume_sum_df)
volume_sum_df.to_csv("C:\\Users\\Ki Siyuan\\Desktop\\volume_sum_df.csv")

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_1448\2418109181.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_sum_df = volume_df.groupby(by=['region', 'year', 'month']).sum()


     Agriculture    Apparel  Chemicals  Machinery  Materials     Metals  \
0       1.221575   1.581593   0.742286   0.156298   0.482838   2.691954   
1       0.953805   1.346038   0.908840   0.119939   0.554936   1.870705   
2       1.950540   1.473313   0.747732   0.124436   0.472334   2.428543   
3       1.905853   0.917737   0.669912   0.045430   0.437803   0.411699   
4       2.392177   1.477671   0.541419   0.139684   0.504537   2.343700   
..           ...        ...        ...        ...        ...        ...   
427    18.114332  44.456603  19.473929  34.478176  27.988578  30.469959   
428    16.554416  42.575248  21.549874  33.005811  26.710469  27.279527   
429    16.290286  44.238275  19.501318  33.679469  26.734570  30.054878   
430    15.900571  42.319798  19.486764  33.476484  26.968659  27.152074   
431    18.018226  44.953763  20.893098  34.211733  27.588302  27.808442   

      Minerals  Miscellaneous  Transport  region  year  month  
0     4.093179       0.491163   0.4

In [4]:
# empty dir
panel_dir = project_dir / 'panels-trade-war'
for fp in panel_dir.rglob('*.csv'):
    os.remove(fp)

In [5]:
exp_years = [2019]
com_years = [2015,2016,2017]
# sector = 'agriculture'
# region = 'China'

# sector = 'agriculture'
# region = 'America'


def get_did_panel(volume_sum_df, exp_years, com_years, sector, region):
    # volume_sum_df 中除去region外的求平均
    volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
    volume_mean_df['year'], volume_mean_df['month'] = zip(*volume_mean_df.index)
    volume_mean_df.reset_index(drop=True, inplace=True)
    
    com_df = volume_mean_df.query('year < 2020')[['year', 'month', sector]]
    com_df['name'] = 'mean'
    com_df.loc[com_df.query('year in @com_years').index, 'time'] = 0
    com_df.loc[com_df.query('year in @exp_years').index, 'time'] = 1
    com_df['time'] = com_df['time'].astype('int')
    com_df.loc[:, 'group'] = 0


    exp_df = volume_sum_df.query('year < 2020 and region == @region')[['region', 'year', 'month', sector]]
    exp_df = exp_df.rename(columns={'region': 'name'})
    exp_df.loc[exp_df.query('year in @com_years').index, 'time'] = 0
    exp_df.loc[exp_df.query('year in @exp_years').index, 'time'] = 1
    exp_df['time'] = exp_df['time'].astype('int')
    exp_df.loc[:, 'group'] = 1


    concat_df = pd.concat([exp_df, com_df])
    concat_df['did'] = concat_df.apply(lambda x: x['time'] * x['group'], axis=1)
    concat_df['date'] = concat_df.apply(lambda x: '{}-{}'.format(x['year'], x['month']), axis=1)
    del concat_df['year'], concat_df['month']
    concat_df = concat_df.rename(columns={sector: 'value'})
    # concat_df['value'] = np.log(concat_df['value'] + 1)
    return concat_df

for region in region2countries_dict.keys():
    for sector in sectors:
        panel_df = get_did_panel(volume_sum_df, exp_years, com_years, sector, region)
        panel_df.to_csv(panel_dir / f'{region}-{sector}.csv', header=True, index=False)

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_14332\192052005.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_14332\192052005.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_14332\192052005.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_o

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_14332\192052005.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_14332\192052005.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_14332\192052005.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_o

### did - trade war

In [6]:
import re
import ipystata

region_list = list(region2countries_dict.keys())
sector_list = sectors

# empty dir
panel_dir = project_dir / 'temp'
for fp in panel_dir.rglob('*.txt'):
    os.remove(fp)

IPyStata is loaded in batch mode.


In [7]:
%%stata -i sector_list -i region_list
foreach region in `region_list' {
    display "------------------------`region'---------------------------------"
    clear
    cd "C:\\shares_TARIFF_project\\comtrade_new\\DID_cancel2018"
    foreach sector in `sector_list' {
        import delimited "./panels-trade-war/`region'-`sector'.csv"
        egen time1 = group(date)
        xtset time1 group
        reghdfe value did, absorb(time1 group)
        outreg2 using "./temp/tradewar-`region'-`sector'.txt"
        clear
    }
}


------------------------Africa---------------------------------
C:\shares_TARIFF_project\comtrade_new\DID_cancel2018
(6 vars, 96 obs)
       panel variable:  time1 (strongly balanced)
        time variable:  group, 0 to 1
                delta:  1 unit
(MWFE estimator converged in 2 iterations)

HDFE Linear regression                            Number of obs   =         96
Absorbing 2 HDFE groups                           F(   1,     46) =       0.01
                                                  Prob > F        =     0.9204
                                                  R-squared       =     0.9990
                                                  Adj R-squared   =     0.9980
                                                  Within R-sq.    =     0.0002
                                                  Root MSE        =     0.3959

------------------------------------------------------------------------------
       value |      Coef.   Std. Err.      t    P>|t|     [95% Conf. 

In [8]:
def get_result(result_fp):
    with open(result_fp, 'r') as fp:
        result_str = fp.read()
        match_str = re.findall(r'did\t(.*?)\n', result_str)[0]
        star_count = match_str.count('*')
        match_str = match_str.replace(',', '')
        if match_str == '-':# 若匹配到空，判定不显著
            significant = -1
            coff = -999
        else:
            if star_count == 0:
                significant = -1
                coff = float(match_str)
            else:
                coff = float(match_str.split('*')[0])
                if star_count == 1:
                    significant = 0.1
                elif star_count == 2:
                    significant = 0.05
                else:
                    significant = 0.01
    return coff, significant

result_list = []
for region in region_list:
    for sector in sector_list:
        result_fp = project_dir / 'temp' / f'tradewar-{region}-{sector}.txt'
        coff, significant = get_result(result_fp)
        result_list.append((region, sector, coff, significant))
result_df = pd.DataFrame(result_list, columns=['region', 'sector', 'coff', 'significance'])



# # 将coff的值标准化
# data = result_df['coff']
# data_t = (data-data.mean())/(data.max()-data.min())
# result_df['coff'] = data_t
result_df.to_csv('C:\\shares_TARIFF_project\\comtrade_new\\DID_cancel2018\\did-summary\\did-result-trade-war-macro.csv', header=True, index=False)
# result_df

### get panel data - pandemic - relative / true

In [31]:
country2region_dict = {}
for rg, ct_list in region2countries_dict.items():
    for ct in ct_list:
        country2region_dict[ct] = rg

# for sector in sectors:
#     volume_df[sector] = np.log(volume_df[sector] + 1)

volume_df['region'] = volume_df['source'].map(country2region_dict)
volume_sum_df = volume_df.groupby(by=['region', 'year', 'month']).sum()

volume_sum_df['region'], volume_sum_df['year'], volume_sum_df['month'] = zip(*volume_sum_df.index)
volume_sum_df.reset_index(drop=True, inplace=True)
volume_sum_df.head()
# volume_mean_df = volume_sum_df.groupby(by=['year', 'month']).mean()
# volume_mean_df['year'], volume_mean_df['month'] = zip(*volume_mean_df.index)
# volume_mean_df.reset_index(drop=True, inplace=True)


# volume => percentage 转化为占比
years = volume_sum_df['year'].unique()
months = volume_sum_df['month'].unique()
for y in years:
    for m in months:
        query_df = volume_sum_df.query('year == @y and month == @m')[sectors]
        _sum = query_df.sum()
        secter2sum_dict = {s:v for s, v in zip(_sum.index, _sum.tolist())}
        for idx in query_df.index:
            for s, v in secter2sum_dict.items():
                _v = volume_sum_df.loc[idx, s]
                p = _v / v * 100
                volume_sum_df.loc[idx, s] = p

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_14332\1608375874.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_sum_df = volume_df.groupby(by=['region', 'year', 'month']).sum()


In [32]:
# empty dir
panel_dir = project_dir / 'panels-pandemic-dual'
for fp in panel_dir.rglob('*.csv'):
    os.remove(fp)

In [33]:
exp_years = [2020,2021]
com_years = [2015,2016,2017]
#com_years = [2019]


def get_did_panel(volume_sum_df, exp_years, com_years, sector, region):
    # volume_sum_df 中除去region外的求平均
    volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
    volume_mean_df['year'], volume_mean_df['month'] = zip(*volume_mean_df.index)
    volume_mean_df.reset_index(drop=True, inplace=True)
    
    com_df = volume_mean_df[['year', 'month', sector]]
    com_df['name'] = 'mean'
    com_df.loc[com_df.query('year in @com_years').index, 'time'] = 0
    com_df.loc[com_df.query('year in @exp_years').index, 'time'] = 1
    com_df = com_df.dropna()
    com_df['time'] = com_df['time'].astype('int')
    com_df.loc[:, 'group'] = 0


    exp_df = volume_sum_df.query('region == @region')[['region', 'year', 'month', sector]]
    exp_df = exp_df.rename(columns={'region': 'name'})
    exp_df.loc[exp_df.query('year in @com_years').index, 'time'] = 0
    exp_df.loc[exp_df.query('year in @exp_years').index, 'time'] = 1
    exp_df = exp_df.dropna()
    exp_df['time'] = exp_df['time'].astype('int')
    exp_df.loc[:, 'group'] = 1


    concat_df = pd.concat([exp_df, com_df])
    concat_df['did'] = concat_df.apply(lambda x: x['time'] * x['group'], axis=1)
    concat_df['date'] = concat_df.apply(lambda x: '{}-{}'.format(x['year'], x['month']), axis=1)
    del concat_df['year'], concat_df['month']
    concat_df = concat_df.rename(columns={sector: 'value'})
    # concat_df['value'] = np.log(concat_df['value'] + 1)
    return concat_df


for region in region2countries_dict.keys():
    for sector in sectors:
        panel_df = get_did_panel(volume_sum_df, exp_years, com_years, sector, region)
        panel_df.to_csv(panel_dir / f'{region}-{sector}.csv', header=True, index=False)

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_14332\138662078.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  volume_mean_df = volume_sum_df.query('region != @region').groupby(by=['year', 'month']).mean()
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_14332\138662078.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_14332\138662078.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_14332\138662078.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['name'] = 'mean'
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_14332\138662078.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df.loc[com_df.query('year in @com_years').index, 'time'] = 0
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_14332\138662078.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_14332\138662078.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df['time'] = exp_df['time'].astype('int')
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_14332\138662078.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df.loc[:, 'group'] = 1
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_14332\138662078.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default

C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_14332\138662078.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df.loc[com_df.query('year in @com_years').index, 'time'] = 0
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_14332\138662078.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_df['time'] = com_df['time'].astype('int')
C:\Users\Ki Siyuan\AppData\Local\Temp\ipykernel_14332\138662078.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

### did - pandemic

In [8]:
import re
import ipystata

region_list = list(region2countries_dict.keys())
sector_list = sectors
print(region_list)

# empty dir
panel_dir = project_dir / 'temp'
for fp in panel_dir.rglob('*.txt'):
    os.remove(fp)

IPyStata is loaded in batch mode.
['Africa', 'America', 'Europe', 'Oceania', 'China', 'ROA']


In [19]:
%%stata -i sector_list -i region_list
foreach region in `region_list' {
    display "------------------------`region'---------------------------------"
    clear
    cd "C:\\shares_TARIFF_project\\comtrade_new\\DID_cancel2018"
    foreach sector in `sector_list' {
        import delimited "./panels-pandemic/`region'-`sector'.csv"
        egen time1 = group(date)
        xtset time1 group
        reghdfe value did, absorb(time1 group)
        outreg2 using "./temp/pandemic-`region'-`sector'.txt"
        clear
    }
}


------------------------Africa---------------------------------
C:\shares_TARIFF_project\comtrade_new\DID_cancel2018
(6 vars, 120 obs)
       panel variable:  time1 (strongly balanced)
        time variable:  group, 0 to 1
                delta:  1 unit
(MWFE estimator converged in 2 iterations)

HDFE Linear regression                            Number of obs   =        120
Absorbing 2 HDFE groups                           F(   1,     58) =       0.37
                                                  Prob > F        =     0.5436
                                                  R-squared       =     0.9991
                                                  Adj R-squared   =     0.9981
                                                  Within R-sq.    =     0.0064
                                                  Root MSE        =     0.3840

------------------------------------------------------------------------------
       value |      Coef.   Std. Err.      t    P>|t|     [95% Conf.

In [20]:
def get_result(result_fp):
    with open(result_fp, 'r') as fp:
        result_str = fp.read()
        match_str = re.findall(r'did\t(.*?)\n', result_str)[0]
        star_count = match_str.count('*')
        match_str = match_str.replace(',', '')
        if match_str == '-':# 若匹配到空，判定不显著
            significant = -1
            coff = -999
        else:
            if star_count == 0:
                significant = -1
                coff = float(match_str)
            else:
                coff = float(match_str.split('*')[0])
                if star_count == 1:
                    significant = 0.1
                elif star_count == 2:
                    significant = 0.05
                else:
                    significant = 0.01
    return coff, significant

result_list = []
for region in region_list:
    for sector in sector_list:
        result_fp = project_dir / 'temp' / f'pandemic-{region}-{sector}.txt'
        coff, significant = get_result(result_fp)
        result_list.append((region, sector, coff, significant))
result_df = pd.DataFrame(result_list, columns=['region', 'sector', 'coff', 'significance'])

# # 将coff的值区间缩放至 -1,1
#data = result_df['coff']
#data_t = (data-data.mean())/(data.max()-data.min())
#result_df['coff'] = data_t
result_df.to_csv('C:\\shares_TARIFF_project\\comtrade_new\\DID_cancel2018\\did-summary\\did-result-pandemic-macro-relative.csv', header=True, index=False)
#result_df.to_csv('C:\\shares_TARIFF_project\\comtrade_new\\DID_cancel2018\\did-summary\\did-result-pandemic-macro-true.csv', header=True, index=False)
result_df

,region,sector,coff,significance
0,Africa,Agriculture,-0.087400,-1.00
1,Africa,Apparel,0.505000,0.01
2,Africa,Chemicals,-0.099900,-1.00
3,Africa,Machinery,-0.017400,0.05
4,Africa,Materials,0.197000,0.01
5,Africa,Metals,1.986000,0.01
6,Africa,Miscellaneous,-0.046400,-1.00
7,Africa,Transport,-0.202000,0.01
8,Africa,Minerals,-3.551000,0.01
9,America,Agriculture,1.273000,0.10


### comparsion between trade war and pandemic

In [22]:
import numpy as np
result_tradewar_fp = 'C:\\shares_TARIFF_project\\comtrade_new\\DID_cancel2018\\did-summary\\did-result-trade-war-macro.csv'
result_tradewar_df = pd.read_csv(result_tradewar_fp)
result_rpandemic_fp = 'C:\\shares_TARIFF_project\\comtrade_new\\DID_cancel2018\\did-summary\\did-result-pandemic-macro-relative.csv'
result_rpandemic_df = pd.read_csv(result_rpandemic_fp)
result_tpandemic_fp = 'C:\\shares_TARIFF_project\\comtrade_new\\DID_cancel2018\\did-summary\\did-result-pandemic-macro-true.csv'
result_tpandemic_df = pd.read_csv(result_tpandemic_fp)



# 如果sign=-1（不显著），设置coff为NaN
def func(x):
    if x['significance'] == -1:
        return np.nan
    else:
        return x['coff']
result_tradewar_df['coff'] = result_tradewar_df.apply(func, axis=1)
result_rpandemic_df['coff'] = result_rpandemic_df.apply(func, axis=1)
result_tpandemic_df['coff'] = result_tpandemic_df.apply(func, axis=1)


merge_df = pd.merge(result_tradewar_df, result_rpandemic_df, 
         left_on=['region', 'sector'], 
         right_on=['region', 'sector'], 
         how='inner') \
    .rename(columns={'coff_x': 'coff_tradewar', 
                     'coff_y': 'coff_rpandemic',
                     'significance_x': 'sign_tradewar', 
                     'significance_y': 'sign_rpandemic'})


merge_df = pd.merge(merge_df, result_tpandemic_df, 
         left_on=['region', 'sector'], 
         right_on=['region', 'sector'], 
         how='inner') \
    .rename(columns={'coff': 'coff_tpandemic', 
                     'significance': 'sign_tpandemic'})



# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# data = merge_df['coff_tradewar'].tolist()
# data_t = scaler.fit_transform(np.array(data).reshape(-1,1)).reshape(1,-1).tolist()[0]
# merge_df['coff_tradewar'] = data_t


# data = merge_df['coff_pandemic'].tolist()
# data_t = scaler.fit_transform(np.array(data).reshape(-1,1)).reshape(1,-1).tolist()[0]
# merge_df['coff_pandemic'] = data_t

merge_df.to_csv('C:\\shares_TARIFF_project\\comtrade_new\\DID_cancel2018\\did-summary\\did-result-merge-macro.csv', header=True, index=False)
merge_df.head()

,region,sector,coff_tradewar,sign_tradewar,coff_rpandemic,sign_rpandemic,coff_tpandemic,sign_tpandemic
0,Africa,Agriculture,NaN,-1.00,NaN,-1.00,NaN,-1.00
1,Africa,Apparel,0.572,0.01,0.5050,0.01,NaN,-1.00
2,Africa,Chemicals,NaN,-1.00,NaN,-1.00,-0.249,0.05
3,Africa,Machinery,NaN,-1.00,-0.0174,0.05,-0.022,0.05
4,Africa,Materials,0.154,0.01,0.1970,0.01,NaN,-1.00


In [23]:
for coff_str in ['coff_tradewar', 'coff_rpandemic', 'coff_tpandemic']:
    pivot_df = merge_df[['region', 'sector', coff_str]].set_index('region') \
        .pivot(columns='sector', values=coff_str)
    pivot_df.to_csv(f'C:\\shares_TARIFF_project\\comtrade_new\\DID_cancel2018\\did-summary\\pivot-{coff_str}-macro.csv', header=True, index=True)

In [33]:
import pyecharts.options as opts
from pyecharts.charts import Bar

data = pd.read_csv(r"C:\\shares_TARIFF_project\\comtrade_new\\DID_cancel2018\\did-summary\\pivot-coff_tradewar-macro.csv")
print(data)

    region  Agriculture  Apparel  Chemicals  Machinery  Materials   Metals  \
0   Africa          NaN    0.572        NaN        NaN      0.154  0.73600   
1  America          NaN      NaN     -3.071      1.379     -1.362      NaN   
2    China       -2.085   -7.710     -1.407     -6.887     -2.830 -1.41400   
3   Europe          NaN    0.888      5.804      2.108      0.550      NaN   
4  Oceania          NaN      NaN        NaN        NaN        NaN -0.00819   
5      ROA        1.575    6.025     -1.474      3.395      3.490      NaN   

   Minerals  Miscellaneous  Transport  
0       NaN            NaN     -0.322  
1       NaN         -3.334      3.428  
2    -0.271         -5.043     -0.331  
3     4.431          5.139     -1.293  
4       NaN            NaN        NaN  
5    -4.191          3.164     -1.482  


In [25]:
import random
import pyecharts.options as opts
from pyecharts.charts import Bar

x_vals = ['Africa', 'America', 'China', 'Europe', 'ROA']


bar = (
    Bar()
    .add_xaxis(x_vals)
    .add_yaxis("Agriculture", [0, 0, -0.71, 0, 0], category_gap="10%")
    .add_yaxis("Apparel", [0.430, 0, -3.225, 0.714,0],category_gap="10%")
    .add_yaxis("Chemical", [0, -4.108, -1.948, 7.431, -1.670],category_gap="10%")
    .add_yaxis("Machinery", [0, 0, 0, 1.374, 0],category_gap="10%")
    .add_yaxis("Materials", [0, -5.187, 0, 0, 4.178],category_gap="10%")
    .add_yaxis("Metals", [0.964, 0, 0, 0, 0],category_gap="10%")
    .add_yaxis("Minerals",[0, 0, -0.130, 2.559],category_gap="10%")
    .add_yaxis("Miscellaneous", [0, -2.180, 0, 3.067, 0],category_gap="10%")
    .add_yaxis("Transport", [0, 0, 0.673, 0, 0],category_gap="10%")
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False, font_size=14))
    .set_global_opts(title_opts=opts.TitleOpts(title="(a)"),
                     xaxis_opts=opts.AxisOpts(name='region',offset=-150))                   
)
bar.render('C:/shares_TARIFF_project/comtrade_new/DID/tradewar.html')


'C:\\shares_TARIFF_project\\comtrade_new\\DID\\tradewar.html'

In [24]:
import pyecharts.options as opts
import pandas as pd
from pyecharts.charts import Bar

data = pd.read_csv(r"C:\\shares_TARIFF_project\\comtrade\\DID\\did-summary\\pivot-coff_rpandemic-macro.csv")
print(data)

    region  Agriculture  Apparel  Chemicals  Machinery  Materials  Metals  \
0   Africa          NaN    0.576        NaN        NaN      0.295   1.968   
1  America          NaN      NaN     -5.700        NaN     -3.894     NaN   
2    China       -2.774  -10.950     -3.048     -9.188     -4.182  -4.996   
3   Europe       -1.190      NaN     10.690     -1.292     -1.473   8.230   
4  Oceania          NaN      NaN        NaN        NaN        NaN     NaN   
5      ROA        2.302    9.151     -1.901      8.447      8.821  -3.069   

   Minerals  Miscellaneous  Transport  
0    -3.359            NaN        NaN  
1     5.258         -5.578        NaN  
2    -0.422         -4.604        NaN  
3     4.496          2.721     -2.363  
4       NaN            NaN        NaN  
5    -5.569          6.680        NaN  


In [25]:
import random
import pyecharts.options as opts
from pyecharts.charts import Bar

x_vals = ['Africa', 'America', 'China', 'Europe', 'ROA']


bar = (
    Bar()
    .add_xaxis(x_vals)
    .add_yaxis("Agriculture", [0, 0, -2.774, -1.190, 2.302], category_gap="10%")
    .add_yaxis("Apparel", [0.576, 0, -10.950, 0, 9.151],category_gap="10%")
    .add_yaxis("Chemical", [0, -5.700, -3.048, 10.690, -1.901],category_gap="10%")
    .add_yaxis("Machinery", [0, 0, -9.188, -1.292, 8.447],category_gap="10%")
    .add_yaxis("Materials", [0.295, -3.894, -4.182, -1.473, 8.821],category_gap="10%")
    .add_yaxis("Metals", [1.968, 0, -4.996, 8.230, -3.069],category_gap="10%")
    .add_yaxis("Minerals",[-3.359, 5.258, -0.422, 4.496, -5.569],category_gap="10%")
    .add_yaxis("Miscellaneous", [0, -5.578, -4.604, 2.721, 6.680],category_gap="10%")
    .add_yaxis("Transport", [0, 0, 0, -2.363, 0],category_gap="10%")
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False, font_size=14))
    .set_global_opts(title_opts=opts.TitleOpts(title="(a)"),
                     xaxis_opts=opts.AxisOpts(name='region',offset=-150))                   
)
bar.render('C:/shares_TARIFF_project/comtrade/DID/rpandemic.html')

'C:\\shares_TARIFF_project\\comtrade\\DID\\rpandemic.html'

In [26]:
import pyecharts.options as opts
import pandas as pd
from pyecharts.charts import Bar

data = pd.read_csv(r"C:\\shares_TARIFF_project\\comtrade\\DID\\did-summary\\pivot-coff_rpandemic-macro.csv")
print(data)

    region  Agriculture  Apparel  Chemicals  Machinery  Materials  Metals  \
0   Africa          NaN    0.576        NaN        NaN      0.295   1.968   
1  America          NaN      NaN     -5.700        NaN     -3.894     NaN   
2    China       -2.774  -10.950     -3.048     -9.188     -4.182  -4.996   
3   Europe       -1.190      NaN     10.690     -1.292     -1.473   8.230   
4  Oceania          NaN      NaN        NaN        NaN        NaN     NaN   
5      ROA        2.302    9.151     -1.901      8.447      8.821  -3.069   

   Minerals  Miscellaneous  Transport  
0    -3.359            NaN        NaN  
1     5.258         -5.578        NaN  
2    -0.422         -4.604        NaN  
3     4.496          2.721     -2.363  
4       NaN            NaN        NaN  
5    -5.569          6.680        NaN  


In [27]:
import pyecharts.options as opts
import pandas as pd
from pyecharts.charts import Bar

data = pd.read_csv(r"C:\\shares_TARIFF_project\\comtrade\\DID\\did-summary\\pivot-coff_tpandemic-macro.csv")
print(data)

    region  Agriculture  Apparel  Chemicals  Machinery  Materials  Metals  \
0   Africa          NaN      NaN        NaN        NaN        NaN   1.004   
1  America        2.489      NaN     -1.592        NaN        NaN     NaN   
2    China       -2.063   -7.724     -1.100     -7.178     -5.148  -4.702   
3   Europe       -1.685      NaN      3.260     -2.665     -1.734   7.629   
4  Oceania          NaN      NaN        NaN        NaN        NaN     NaN   
5      ROA        1.687    6.899        NaN      7.869      4.642  -2.473   

   Minerals  Miscellaneous  Transport  
0    -2.967            NaN        NaN  
1     6.744         -3.398        NaN  
2    -0.292            NaN        NaN  
3     1.937            NaN        NaN  
4       NaN            NaN        NaN  
5    -4.822          5.690        NaN  
